# 问题的提出

## 欧拉猜想

任何一个正整数的 $n$ 次方，至少需要 $n$ 个正整数的 $n$ 次方才能表示出来。

寻找 $n = 5$ 时的反例：

$x^5 + y^5 + z^5 + u^5 = w^5$

# 计算方法

## 1. 简单枚举法

这类问题的一般解法是枚举法，也就是通常说的“暴力破解”。比如，我们可以用以下方法写一个最简单的实现。

In [27]:
def power_sum_v1(nmax):
    ''' 寻找方程 x**5 + y**5 + z**5 + u**5 = w**5
        在 1 <= x <= y <= z <= u < w < nmax 内的正数解 '''
    
    for x in range(1, nmax):
        for y in range(x, nmax):
            for z in range(y, nmax):
                for u in range(z, nmax):
                    s = x**5 + y**5 + z**5 + u**5
                    for w in range(u, nmax):
                        if s == w**5:
                            print(x, y, z, u, w)

In [30]:
%timeit power_sum_v1(100)

22.3 s ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


注意到，

* 我们这里用了 `%timeit` 来测试这个程序的运行时间，这个命令会调用这个函数好几遍，运行后，计算平均时间。

* 同时注意到，我们在计算中并没有用 `nmax=200` 而是用了一个比较小的值 `nmax=100`，因为我们在开发算法的过程中，会经常用一些比较小的值，让程序尽快运行完，这样好研究算法的效率。

即便是 `nmax=100` 我们的程序也是异常的慢。我的机器上平均每次运行花了 22 秒的时间，加上 `%timeit` 函数运行了 7 编，就是跑了2分多钟才跑完。所以我们要想想有没有办法让它跑的更快点。

## 2. 备忘录方法（查表法）

我们注意到在这个过程中，我们计算了很多次自然数的5次方。这些计算都是重复和没有必要的。所以我们可以事先把1到200内自然数的5次方计算好，列张表。然后以后要用时，直接查这张表就好了。

在 Python 里这样一个表可以用列表 list 来实现。我们把这个表叫做 `pow5`，我们可以这么写来构造这张表：

```
pow5 = []
for x in range(nmax):
    pow5[x] = x**5
```

Python 提供了一种更加简洁的语法来实现这个循环。这种语法叫做 list comprehension，其实就是一种简写循环的方式

```
pow5 = [x**5 for x in range(nmax)]
```


列表以后还有个好处，就是我们可以省掉最内的一层循环。为什么呢？因为如果 $s = x^5+y^5+z^5+u^5$ 是一个数的5次方，那么它必然也在这个表内，所以我们可以用

```
s in pow5
```

来判断 `s` 是否是一个数的5次方。

In [33]:
def power_sum_v2(nmax):
    ''' 寻找方程 x**5 + y**5 + z**5 + u**5 = w**5
        在 1 <= x <= y <= z <= u < w < nmax 内的正数解 '''

    # 把 x = 1..nmax-1 的 x**5 的值做一个表格
    pow5 = [x**5 for x in range(nmax)]
        
    for x in range(1, nmax):
        for y in range(x, nmax):
            for z in range(y, nmax):
                for u in range(z, nmax):
                    s = pow5[x] + pow5[y] + pow5[z] + pow5[u]
                    if s in pow5:
                        print(x, y, z, u, s**(1/5))

In [34]:
%timeit power_sum_v2(100)

3.54 s ± 29.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


我们发现，用了查表法，程序一下子快了很多。在我的机器上，平均每次只用了约 3.5s 的时间。比起上次的 22s 可好多了。

## 3. 更好的查表法（字典）

其实我们判断$s$是否是整数那步可以更有效。原因是如果 `pow5` 是一个列表 `list`，Python 在执行

```
s in pow5
```

它仍然需要把这个列表里每个数字查一遍，才能判断 `s` 是否在列表 `pow5` 内，所以它的算法复杂度是 $O(n)$，其中 $n$ 代表列表里元素的个数。

但我们想想，假定我们有另外一个数组`t`，这个数组是以 `s` 作为其下标的，`t[s]` 的值是一般是0，但是如果 $s$ 是5次方数，那么`t[s]=1`。这样我们就可以一下判断 $s$ 是不是一个5次方数了对吧？

可惜的是，这种数组虽然存在，但是它的内存占用量太大了。200^5是个天文数字，根本无法弄那么大的数组。

所幸的是， Python 还提供了一种更有效的数据结构，叫做字典，可以处理这种稀疏的数组。

```
t = {}
for w in range(nmax):
    t[w**5] = 1
```

对于一个五次方数

```
s in t
```

的值为 `true`，反之为`false`。

注意到，字典和列表用起来很像，只是字典初始化时用 `{}` 而不是 `[]`。

实际实现时，我们可以把字典的值映射为 w，而不是简单的 1，这样就可以根据 s 迅速反查 w

```
invpow5 = {}
for w in range(nmax):
    invpow5[w**5] = 1
```

和列表一样，我们有一种简写的方法：

```
invpow5 = {w**5:w for w in range(1, nmax)}
```

In [36]:
def power_sum_v3(nmax):
    ''' 寻找方程 x**5 + y**5 + z**5 + u**5 = w**5
        在 1 <= x <= y <= z <= u < w < nmax 内的正数解 '''

    # 把 x**5 的值做一个表格
    pow5 = [x**5 for x in range(nmax)]
        
    # 把 w**5 -> w 也做一个反查的表格
    invpow5 = {w**5 : w for w in range(1, nmax)}

    for x in range(1, nmax):
        for y in range(x, nmax):
            for z in range(y, nmax):
                for u in range(z, nmax):
                    s = pow5[x] + pow5[y] + pow5[z] + pow5[u]
                    if s in invpow5:
                        print(x, y, z, u, invpow5[s])

In [37]:
%timeit power_sum_v3(100)

413 ms ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


用了字典以后，程序又快了很多。在我的机器上，平均每次只用了约 0.41s 的时间。比上次的 3.5s 又好很多。

## 4. 循环优化

下一步的优化是对循环优化。

我们看到，在最内的循环里，这句话

```
    for u in range(z, nmax):
        s = pow5[x] + pow5[y] + pow5[z] + pow5[u]
```

会重复计算前三项  `pow5[x] + pow5[y] + pow5[z]` 的值，然而这个值在对 `u` 的循环里循环过过程中是不变的，所以可以把这个量预先计算后提出循环。

In [24]:
def power_sum_v4(nmax):
    ''' 寻找方程 x**5 + y**5 + z**5 + u**5 = w**5
        在 1 <= x <= y <= z <= u < w < nmax 内的正数解 '''

    # 把 x**5 的值做一个表格
    pow5 = [x**5 for x in range(nmax)]
        
    # 把 w**5 -> w 也做一个表格
    invpow = {}
    for w in range(1, nmax):
        invpow[w**5] = w

    for x in range(1, nmax):
        for y in range(x, nmax):
            for z in range(y, nmax):
                s_xyz = pow5[x] + pow5[y] + pow5[z]
                for u in range(z, nmax):
                    s = s_xyz + pow5[u]
                    if s in invpow:
                        print(x, y, z, u, invpow[s])

In [17]:
%timeit power_sum_v4(100)

226 ms ± 16.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


这次只要0.22秒了，又比前一次快了很多。

## 5. 正式运行

经过数轮优化我们终于可以运行真正例子了。

In [26]:
%timeit power_sum_v4(200)

27 84 110 133 144
27 84 110 133 144
27 84 110 133 144
27 84 110 133 144
27 84 110 133 144
27 84 110 133 144
27 84 110 133 144
27 84 110 133 144
3.45 s ± 26.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


这次它真的跑出一个结果，而且它很快，在我的机器上，每次只花3秒钟，真的很快。

## 6. 对照文献结果

让我们对比一下论文的结果：

![Counterexample to Euler's conjecture on sums of like powers](https://i.niupic.com/images/2022/01/24/9Uc9.webp)

完全相同！

# 参考阅读

[这是世界上最短的数学论文之一，关于费马大定理和欧拉猜想](https://mp.weixin.qq.com/s/Pd6R_GynMTRL1_n58ja5og)